## 3.5 Multi Query Generation

In [10]:
# Step1. API KEY 설정
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['OPEANAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [11]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5)

In [12]:
from llama_index.core.prompts import Prompt
# 쿼리 변환용 프롬프트 설정
decompose_prompt = Prompt(
    """
    다음 [Original Query]와 관련있는 여러개의 쿼리를 3가지로 변환시켜주세요.
    출력 포맷은 아래와 같은 형태를 지켜주세요.
    
    Original Query: {query_str}
    Sub-Queries:
    1.
    2.
    3.
    """
)

In [13]:
# DecomposeQueryTransform 초기화
from llama_index.core.indices.query.query_transform.base import DecomposeQueryTransform
decompose = DecomposeQueryTransform(decompose_query_prompt=decompose_prompt, 
                                    llm=Settings.llm,
                                    verbose=True)

In [14]:
query = "RAG에 대해 알려주세요." # 입력 쿼리
query_bundle = decompose.run(query) # 쿼리 변환 수행
print(query_bundle.query_str) # 결과 출력

> Current query: RAG에 대해 알려주세요.
> New query: Original Query: RAG에 대해 알려주세요.
Sub-Queries:
1. RAG의 주요 기능은 무엇인가요?
2. RAG의 장점과 단점은 무엇인가요?
3. RAG의 활용 사례에는 어떤 것들이 있나요?
Original Query: RAG에 대해 알려주세요.
Sub-Queries:
1. RAG의 주요 기능은 무엇인가요?
2. RAG의 장점과 단점은 무엇인가요?
3. RAG의 활용 사례에는 어떤 것들이 있나요?


In [6]:
sub_query = query_bundle.query_str.split('\n')[2:]
input_query = query_bundle.query_str.split('\n')[0]
sub_query_join = '\n'.join(sub_query) # sub query 만 추출

In [21]:
from llama_index.core.prompts import PromptTemplate
final_prompt_template = PromptTemplate(
                        template = """아래 [Original-Query]와 [Sub-Queries]에 대한 내용을 모두 종합 및 요약해서 알려주세요.
                        \n{input_query}
                        \n[Sub-Queires]
                        \n{sub_query}
                        """,
                        template_vars="input_query, sub_query"
                    )
final_prompt = final_prompt_template.format(input_query=input_query,
                                            sub_query=sub_query_join)


In [22]:
print(final_prompt)

아래 [Original-Query]와 [Sub-Queries]에 대한 내용을 모두 종합 및 요약해서 알려주세요.
                        
Original Query: RAG에 대해 알려주세요.  
                        
[Sub-Queires]
                        
1. RAG의 주요 기능과 특징은 무엇인가요?  
2. RAG가 다른 기술과 비교했을 때의 장점은 무엇인가요?  
3. RAG의 활용 사례에는 어떤 것들이 있나요?  
                        


In [3]:
llm = OpenAI(model="gpt-4o", temperature=0.5, max_toknes=300)

In [9]:
## Original qeury response
response_original = llm.complete(query) # original query에 대한 답변
print(response_original)

RAG는 "Retrieval-Augmented Generation"의 약자로, 자연어 처리(NLP) 분야에서 정보를 생성하는 데 있어 검색을 활용하는 방법론을 의미합니다. RAG는 두 가지 주요 구성 요소로 이루어져 있습니다: 검색기(retriever)와 생성기(generator).

1. **검색기(Retriever)**: 이 구성 요소는 주어진 질문이나 프롬프트와 관련된 정보를 대규모 데이터베이스나 문서 집합에서 검색합니다. 보통 이는 BM25, DPR(Dense Passage Retrieval) 같은 기술을 사용하여 관련 문서를 효율적으로 찾아내는 역할을 합니다.

2. **생성기(Generator)**: 검색기를 통해 얻은 정보를 바탕으로, 자연어로 응답을 생성합니다. 이는 주로 Transformer 기반의 언어 모델, 예를 들어 BERT, GPT 등의 모델을 활용하여 보다 자연스럽고 문맥에 맞는 응답을 생성합니다.

RAG의 주요 장점은 대규모의 외부 지식을 활용하여 보다 정확하고 풍부한 정보를 제공할 수 있다는 점입니다. 이는 특히 정보의 정확성이 중요한 분야에서 유용하게 사용될 수 있습니다. 예를 들어, 고객 지원 시스템, 지식 기반 시스템 등에서 RAG를 활용하여 사용자가 필요로 하는 정보를 보다 정확하게 제공할 수 있습니다.


In [10]:
## Multi query response
response = llm.complete(final_prompt)

In [12]:
print(response.text)

RAG(Recursively Augmented Generation)은 인공지능 분야에서 주로 사용되는 기법으로, 대규모 언어 모델과 검색 시스템을 결합하여 정보를 생성하고 제공하는 데 활용됩니다. 다음은 RAG에 대한 종합적인 요약입니다.

1. **주요 기능과 특징**:
   - RAG는 대규모 언어 모델(GPT-3 등)과 정보 검색 시스템을 결합하여 보다 정확하고 신뢰성 있는 응답을 생성합니다.
   - 검색 시스템은 외부 데이터베이스나 문서에서 관련 정보를 검색하고, 언어 모델은 이를 바탕으로 자연스러운 문장을 생성합니다.
   - 이 방법은 단순히 언어 모델만을 사용하는 것보다 최신 정보와 정확한 데이터를 제공하는 데 유리합니다.

2. **장점**:
   - **정확성 향상**: 외부 데이터베이스에서 직접 정보를 가져오기 때문에 최신성과 정확성이 높습니다.
   - **확장성**: 다양한 분야의 정보를 쉽게 통합할 수 있어, 특정 분야에 국한되지 않고 광범위한 주제를 다룰 수 있습니다.
   - **유연성**: 사용자가 원하는 정보에 맞춰 검색과 생성을 조합할 수 있어 다양한 요구에 대응할 수 있습니다.

3. **활용 사례**:
   - **고객 지원**: 고객의 질문에 대한 정확한 응답을 제공하기 위해 RAG를 활용하여 실시간으로 관련 정보를 검색하고 응답을 생성합니다.
   - **연구 및 학습**: 연구자나 학생들이 특정 주제에 대한 최신 논문이나 자료를 빠르게 검색하고 요약하는 데 사용됩니다.
   - **콘텐츠 생성**: 뉴스 기사나 블로그 포스트 작성 시, 관련 정보를 검색하고 이를 바탕으로 자연스러운 문장을 생성하여 콘텐츠를 작성합니다.

RAG는 이러한 특성과 장점을 바탕으로 다양한 산업과 분야에서 정보를 보다 효과적으로 처리하고 활용하는 데 기여하고 있습니다.
